In [ ]:
command = f"finetune_ldm_decoder.py --num_keys 1 \
    --ldm_config /path/stable-diffusion-1-5-original/v1-inference.yaml \
    --ldm_ckpt /path/stable-diffusion-1-5-original/v1-5-pruned.ckpt \
    --msg_decoder_path ./dec_48b_whit.torchscript.pt \
    --train_dir /path/dataset/wikiart \
    --val_dir ./data1"
%run -i $command

In [ ]:
import torch 
import sys

device = torch.device("cuda")

from omegaconf import OmegaConf 
from diffusers import StableDiffusionPipeline 
from utils_model import load_model_from_config


# loading the pipeline, and replacing the decode function of the pipe
pipe = StableDiffusionPipeline.from_pretrained("/path/stable-diffusion-2-1-base")
pipe = pipe.to("cuda")

img = pipe("a painting by cityscape").images[0]
img.save("./cityscape1.png")


In [ ]:
import torch 
import sys
import os

device = torch.device("cuda")

from omegaconf import OmegaConf 
from diffusers import StableDiffusionPipeline 
from utils_model import load_model_from_config

sys.path.append('src')
from ldm.models.autoencoder import AutoencoderKL
from ldm.models.diffusion.ddpm import LatentDiffusion
from loss.loss_provider import LossProvider

ldm_config = "/path/stable-diffusion-2-1-base-original/v2-inference.yaml"
ldm_ckpt = "/path/stable-diffusion-2-1-base-original/v2-1_512-ema-pruned.ckpt"

print(f'>>> Building LDM model with config {ldm_config} and weights from {ldm_ckpt}...')
config = OmegaConf.load(f"{ldm_config}")
ldm_ae = load_model_from_config(config, ldm_ckpt)
ldm_aef = ldm_ae.first_stage_model
ldm_aef.eval()

# loading the fine-tuned decoder weights
state_dict = torch.load("./output/checkpoint_000.pth")
unexpected_keys = ldm_aef.load_state_dict(state_dict, strict=False)
print(unexpected_keys)
print("you should check that the decoder keys are correctly matched")


pipe = StableDiffusionPipeline.from_pretrained("/path/stable-diffusion-2-1-base", safety_checker=None)
pipe = pipe.to("cuda")
pipe.vae.decode = (lambda x,  *args, **kwargs: ldm_aef.decode(x).unsqueeze(0))

img = pipe(f"the cat drinks water.").images[0]
img.save(f"cat.png")


In [ ]:
import subprocess


!python ./run_evals.py --eval_imgs False --eval_bits True \
    --img_dir "./attack_images/artist/10_resize_restore" \
    --key_str '111010110101000001010111010011010100010000100111'

import pandas as pd
data = pd.read_csv("./output/log_stats.csv")
print(data["bit_acc_none"].mean())

In [ ]:
artists = ['albrecht-durer', 'childe-hassam', 'claude-monet', 'edgar-degas', 'eugene-boudin', 'ilya-repin',
           'john-singer-sargent', 'pierre-auguste-renoir', 'rembrandt', 'vincent-van-gogh']
concepts = ['decoritems_houseplant', 'furniture_chair', 'jewelry_earring', 'luggage_purse', 'person', 'plushie_panda',
           'things_bottle', 'things_headphone', 'transport_car', 'wearable_sunglasses']
people = ['Donald_Trump', 'Elon_Reeve_Musk', 'Jackie_Chan', 'Leonardo_DiCaprio', 'Taylor_Alison_Swift']
import os
import shutil

folder = "./outputs/artist"
for sub in os.listdir(folder):
    subfoler = folder + "/" + sub + "/"
    for n in os.listdir(subfoler):
        shutil.copy2(subfoler + n, "./watermarked_image/artist" + f"/{sub}_{n}")